In [3]:
import numpy as np
import os
import cv2
import tensorflow as tf
import tensorflow.keras as K
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

In [22]:
TRAIN_DIR = './train'
VAL_DIR = './val'
EPOCHS = 10
imageSize = 300

In [23]:
trainImageGen = ImageDataGenerator(rescale = 1./255.)
valImageGen = ImageDataGenerator(rescale = 1./255.)

In [24]:
trainingSet = trainImageGen.flow_from_directory(batch_size = 64,
                                                directory = TRAIN_DIR,
                                                shuffle = True,
                                                target_size = (imageSize, imageSize),
                                                class_mode = 'binary'
                                               )

valSet = valImageGen.flow_from_directory(batch_size = 64,
                                         directory = VAL_DIR,
                                         shuffle = False,
                                         target_size = (imageSize, imageSize),
                                         class_mode = 'binary'
                                        )

Found 1376 images belonging to 2 classes.
Found 142 images belonging to 2 classes.


In [25]:
print(trainingSet.class_indices)

{'with_mask': 0, 'without_mask': 1}


In [32]:
def getMaskModel():
    K.backend.clear_session()
    
    inputs = layers.Input(shape = (imageSize, imageSize, 3))
    
    model = layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3))(inputs)
    model = layers.MaxPooling2D(2, 2)(model)
    model = layers.Conv2D(32, (3,3), activation='relu')(model)
    model = layers.MaxPooling2D(2,2)(model)
    model = layers.Conv2D(64, (3,3), activation='relu')(model)
    model = layers.MaxPooling2D(2,2)(model)
    model = layers.Conv2D(64, (3,3), activation='relu')(model)
    model = layers.MaxPooling2D(2,2)(model)
    model = layers.Conv2D(64, (3,3), activation='relu')(model)
    model = layers.MaxPooling2D(2,2)(model)
    model = layers.Flatten()(model)
    model = layers.Dense(512, activation='relu')(model)
    model = layers.Dense(1, activation='sigmoid')(model)
    
    maskModel = K.models.Model(inputs = inputs, outputs = model)
    
    return maskModel

In [33]:
model = getMaskModel()

In [34]:
opt = K.optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)
model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

In [35]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0     

In [36]:
history = model.fit(trainingSet,
                    steps_per_epoch =  1376 // 64,
                    epochs = EPOCHS,
                    validation_data = valSet,
                    validation_steps = 142 // 64
                   )

Epoch 1/10
21/21 [==============================] - 113s 5s/step - loss: 0.4102 - accuracy: 0.8018 - val_loss: 0.1276 - val_accuracy: 0.9531
Epoch 2/10
21/21 [==============================] - 92s 4s/step - loss: 0.1182 - accuracy: 0.9665 - val_loss: 0.0789 - val_accuracy: 0.9688
Epoch 3/10
21/21 [==============================] - 91s 4s/step - loss: 0.0781 - accuracy: 0.9748 - val_loss: 0.0877 - val_accuracy: 0.9766
Epoch 4/10
21/21 [==============================] - 93s 4s/step - loss: 0.0637 - accuracy: 0.9802 - val_loss: 0.0251 - val_accuracy: 0.9922
Epoch 5/10
21/21 [==============================] - 92s 4s/step - loss: 0.0709 - accuracy: 0.9802 - val_loss: 0.0640 - val_accuracy: 0.9844
Epoch 6/10
21/21 [==============================] - 96s 5s/step - loss: 0.0740 - accuracy: 0.9779 - val_loss: 0.0562 - val_accuracy: 0.9844
Epoch 7/10
21/21 [==============================] - 93s 4s/step - loss: 0.0296 - accuracy: 0.9893 - val_loss: 0.0515 - val_accuracy: 0.9844
Epoch 8/10
21/21 [=

In [39]:
model.save_weights("mask-detect-model-weights.h5")

In [40]:
with open('mask-detect-model.json', 'w+') as file:
    file.write(model.to_json())